In [ ]:
import json
import random
import warnings

import fastjsonschema
import hypernetx as hnx
import numpy as np
import pandas as pd

In [ ]:
warnings.simplefilter("ignore")

## Create a hypergraph with data

In [ ]:
from hypernetx.utils import toys

lesmis = toys.LesMis()
names = lesmis.df_names
scenes = lesmis.df_scenes
scenes["edges"] = [
    ".".join([str(scenes.loc[idx][col]) for col in scenes.columns[:-2]])
    for idx in scenes.index
]
scenes["nodes"] = scenes["Characters"]
scenes

In [ ]:
df = scenes[["edges", "nodes"]]
cell_weights = df.groupby(["edges"]).count().to_dict()["nodes"]
df["weights"] = df.edges.map(lambda e: np.round(1 / cell_weights.get(e, 1), 2))
df

In [ ]:
nprops = names
nprops["weights"] = np.round(np.random.uniform(0, 1, len(names)), 2)
nprops

In [ ]:
lm = hnx.Hypergraph(
    df,
    cell_weight_col="weights",
    node_properties=nprops,
    node_weight_prop_col="weights",
)
lm.dataframe

In [ ]:
lm.nodes.dataframe

## Load schema and create validator

In [ ]:
schema = json.load(open("hif_schema_v0.1.0.json","r"))
schema

In [ ]:
validator = fastjsonschema.compile(schema);

## HNX translators

In [ ]:
def to_hif(hg):
    edgj = hg.edges.to_dataframe
    edid = edgj.index._name or "index"
    nodj = hg.nodes.to_dataframe
    ndid = nodj.index._name or "index"
    edgj = edgj.reset_index().rename(columns={edid: "edge"}).to_dict(orient="records")
    nodj = nodj.reset_index().rename(columns={ndid: "node"}).to_dict(orient="records")
    incj = (
        hg.incidences.to_dataframe.reset_index()
        .rename(columns={"nodes": "node", "edges": "edge"})
        .to_dict(orient="records")
    )
    hif = {"edges": edgj, "nodes": nodj, "incidences": incj}
    return hif


def from_hif(hif):
    edges = pd.DataFrame(hif["edges"])
    nodes = pd.DataFrame(hif["nodes"])
    incidences = pd.DataFrame(hif["incidences"])
    return hnx.Hypergraph(incidences, node_properties=nodes, edge_properties=edges)

In [ ]:
# %%timeit #5.55 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
hif = to_hif(lm)
json.dump(hif,open("../examples/lesmis_hif.json","w"),allow_nan=False)

In [ ]:
# %%timeit #742 µs ± 2.52 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
try:
    validator(hif);
except Exception as e:
    print(e)

## Simple Testing

In [ ]:

from copy import deepcopy
hiftest = deepcopy(hif)
hiftest['network-type'] = "ordered"
try:
    validator(hiftest);
except Exception as e:
    print(e)
validator(hiftest);

In [ ]:
# %%timeit #9.99 ms ± 219 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
h = from_hif(hif)

In [ ]:
h.dataframe[:5]

## Hypergraph Examples

### contacts-high-school

In [ ]:
hs = json.load(open(f"../examples/contacts-high-school.json", "r"))

In [ ]:
hs.keys()

In [ ]:
hs["hyperedges"][:5]

In [ ]:
hs["nodes"][:2]

In [ ]:
df = pd.DataFrame(hs["hyperedges"]).fillna("")
df["edge"] = df.interaction.map(lambda x: x[0])
df["node"] = df.interaction.map(lambda x: x[1])
df = df[["edge", "node", "time"]]

nodedf = pd.DataFrame(hs["nodes"])
nodedf = nodedf.set_index("id").reset_index().fillna("")

hshyp = hnx.Hypergraph(df, node_properties=nodedf)

In [ ]:
hshyp.dataframe

In [ ]:
hshyp.nodes.dataframe

In [ ]:
hnx.info_dict(hshyp)

In [ ]:
# %%timeit #18.1 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
hshif = to_hif(hshyp)

In [ ]:
try:
    validator(hshif);
except Exception as e:
    print(e)

In [ ]:
# %%timeit #26.2 ms ± 516 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
newhshyp = from_hif(hshif)

In [ ]:
hnx.info_dict(newhshyp)

In [ ]:
assert hshyp == newhshyp ## this checks that the structure is the same

In [ ]:
json.dump(hshif,open("../examples/contacts_high_school_hif.json","w"),allow_nan=False)